# NeRFの推論 on Google Colab

## 必要なモジュールのインストール

In [ ]:
!pip install pyntcloud

In [ ]:
!git clone https://github.com/ALBERT-Inc/NeRF-tutorial.git
%cd "./NeRF-tutorial"
!pip install --editable ./
%cd "../"
import site
site.main()

## ロード/ディレクトリ設定

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from nerf_tutorial.intrinsics import Intrinsic
from nerf_tutorial.extrinsics import PoseExtrinsic, RVecExtrinsic
from nerf_tutorial.nerf import NeRFConfig, NeRF, NeRFLoss
from nerf_tutorial.visualization_utils import IpywidgetsRenderer
from nerf_tutorial.nerf_utils import render_nerf, extract_pointcloud

In [ ]:
base_dir = os.getcwd()
out_dir = os.path.join(base_dir, "results")
ckpt_path = os.path.join(out_dir, "ckpt")

if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)

In [ ]:
# download checkpoint file
!wget "https://github.com/ALBERT-Inc/NeRF-tutorial/blob/main/results/ckpt?raw=true" -O $ckpt_path

## NeRFの呼び出し

In [ ]:
device = torch.device("cuda:0")

# NeRF
config = NeRFConfig()
nerf = NeRF(**config.nerf_kwargs())

# camera parameters
ckpt = torch.load(
    ckpt_path, map_location="cpu")["state_dict"]
pose = ckpt["extrinsic.pose"].numpy()
image_wh = ckpt["intrinsic.image_wh"].numpy()
fs = ckpt["intrinsic.fs"].numpy()
cxcy = ckpt["intrinsic.cxcy"].numpy()

intrinsic = Intrinsic(
    image_wh, focals=fs, cxcy=cxcy, **config.intrinsic_kwargs())

if config.nerfmm:
    extrinsic = RVecExtrinsic(
        len(pose), pose, **config.extrinsic_kwargs())
else:
    extrinsic = PoseExtrinsic(
        len(pose), pose, **config.extrinsic_kwargs())

loss_func = NeRFLoss(nerf, intrinsic=intrinsic, extrinsic=extrinsic)
loss_func.load_state_dict(ckpt)
loss_func.to(device)

## レンダリング

In [ ]:
idx = 0
camera_parameters = {
    "w": 300,
    "h": 200,
    "pose": extrinsic[[idx]],
    "device": device,
}

W, H = image_wh
cx, cy = cxcy
fx, fy = fs

camera_parameters["cx"] = cx / W * camera_parameters["w"]
camera_parameters["cy"] = cy / H * camera_parameters["h"]
camera_parameters["fx"] = fx / W * camera_parameters["w"]
camera_parameters["fy"] = fy / H * camera_parameters["h"]

In [ ]:
# render single image
img = render_nerf(nerf, camera_parameters, only_coarse=False)
plt.imshow(img)
plt.show()

In [ ]:
# renderer with controller
IpywidgetsRenderer(nerf, camera_parameters, only_coarse=True)

## 点群抽出

In [ ]:
pcd = extract_pointcloud(
    nerf, num_grid_edge=100, sigma_threshold=50, device=device)
pcd.to_file(os.path.join(out_dir, "nerf_repro.ply"))